In [45]:
# Add the file to the staging area
!git add PredictNHL.ipynb

# Commit the file with a descriptive message
!git commit -m "Add PredictNHL.ipynb"

# Push the file to the main branch on GitHub
!git push origin main


[main 6896a62] Add PredictNHL.ipynb
 1 file changed, 809 insertions(+), 21 deletions(-)


Auto packing the repository in background for optimum performance.
See "git help gc" for manual housekeeping.
To https://github.com/nic01as1/NHLPredictor.git
   86929b3..6896a62  main -> main


In [1]:
!pip install tensorflow
!pip install xgboost
!pip install streamlit
!pip install imbalanced-learn
!pip install shap
!pip install statsmodels



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import os
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import cross_val_score
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from sklearn.metrics import f1_score, make_scorer, classification_report
from sklearn.metrics import f1_score

In [12]:
#AJOUTER LE NOMBRE DE JOURS DE REPOS !

In [13]:

# URL of the website
url = 'https://www.hockey-reference.com/leagues/NHL_2025.html#all_stats'

# Set up the Selenium WebDriver (e.g., Chrome)
driver = webdriver.Chrome()  # Make sure you have the ChromeDriver installed

# Open the URL
driver.get(url)

# Wait for the page to load
time.sleep(2)  # Adjust the sleep time if necessary

# Get the page source and parse it with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Close the WebDriver
driver.quit()

# Find the "Team Statistics" table by its id
table = soup.find('table', id='stats')

# Check if the table is found
if table is None:
    print("Team Statistics table not found")
else:
    # Create lists to store the scraped data
    teams = []
    ranks = []

    # Iterate through each row of the table
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')

        # Check if the row has the expected number of columns
        if len(columns) >= 2:
            team = columns[0].find('a').text.strip() if columns[0].find('a') else columns[0].text.strip()
            rank = len(teams) + 1  # The rank is the order in which the teams are listed

            # Append the data to the lists
            teams.append(team)
            ranks.append(rank)

    # Create a DataFrame from the lists
    data = {
        'Team': teams,
        'Rank': ranks,
        'Team Rank': [f"{team} {rank}" for team, rank in zip(teams, ranks)]
    }
    df_rank = pd.DataFrame(data)

    # Print the DataFrame
    print(df_rank)

                     Team  Rank                 Team Rank
0     Washington Capitals     1     Washington Capitals 1
1           Winnipeg Jets     2           Winnipeg Jets 2
2    Vegas Golden Knights     3    Vegas Golden Knights 3
3          Minnesota Wild     4          Minnesota Wild 4
4         Edmonton Oilers     5         Edmonton Oilers 5
5     Toronto Maple Leafs     6     Toronto Maple Leafs 6
6       New Jersey Devils     7       New Jersey Devils 7
7     Carolina Hurricanes     8     Carolina Hurricanes 8
8            Dallas Stars     9            Dallas Stars 9
9      Colorado Avalanche    10     Colorado Avalanche 10
10       Florida Panthers    11       Florida Panthers 11
11      Los Angeles Kings    12      Los Angeles Kings 12
12         Calgary Flames    13         Calgary Flames 13
13    Tampa Bay Lightning    14    Tampa Bay Lightning 14
14      Vancouver Canucks    15      Vancouver Canucks 15
15          Boston Bruins    16          Boston Bruins 16
16  Columbus B

In [14]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Function to scrape fixturedownload.com for games history (Date/Home Team/Away Team/Result) - Requests
def scrape_game_history(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table")

    if table is None:
        print(f"Table not found on {url}")
        return [], [], [], [], [], [], []
    else:
        dates = []
        home_teams = []
        away_teams = []
        results = []
        home_scores = []
        away_scores = []
        home_wins = []

        for row in table.find_all("tr")[1:]:  # Skip the header row
            columns = row.find_all("td")

            date = columns[1].text.strip()
            home_team = columns[3].text.strip()
            away_team = columns[4].text.strip()
            result = columns[5].text.strip()

            if any(char.isdigit() for char in result):
                # Split result
                home_score, away_score = result.split("-")

                dates.append(date)
                home_teams.append(home_team)
                away_teams.append(away_team)
                results.append(result)
                home_scores.append(home_score.strip())
                away_scores.append(away_score.strip())

                # Check if Home Team won
                home_win = int(home_score.strip()) > int(away_score.strip())
                home_wins.append(home_win)
            else:
                # Handle cases where the result is not valid
                dates.append(date)
                home_teams.append(home_team)
                away_teams.append(away_team)
                results.append(result)
                home_scores.append(None)
                away_scores.append(None)
                home_wins.append(None)

        return dates, home_teams, away_teams, results, home_scores, away_scores, home_wins


# URLs for scraping
url_history = "https://fixturedownload.com/results/nhl-2024"
#url_ranking = "https://www.hockey-reference.com/leagues/NHL_2025.html#stats"

# Scrape data using requests and Selenium
dates, home_teams, away_teams, results, home_scores, away_scores, home_wins = scrape_game_history(url_history)


# Print the lengths of the lists for debugging
print(f"Dates: {len(dates)}, Home Teams: {len(home_teams)}, Away Teams: {len(away_teams)}, Results: {len(results)}, Home Scores: {len(home_scores)}, Away Scores: {len(away_scores)}, Home Wins: {len(home_wins)}")

# Create a DataFrame from the lists
data = {
    "Date": dates,
    "Home Team": home_teams,
    "Home Score": home_scores,
    "Away Team": away_teams,
    "Away Score": away_scores,
    "Result": results,
    "Home Win": home_wins
}

df_game = pd.DataFrame(data)

# Remove the hour from the dates
df_game["Date"] = df_game["Date"].apply(lambda x: x.split(" ")[0])

print(df_game)


Dates: 1312, Home Teams: 1312, Away Teams: 1312, Results: 1312, Home Scores: 1312, Away Scores: 1312, Home Wins: 1312
            Date              Home Team Home Score            Away Team  \
0     04/10/2024         Buffalo Sabres          1    New Jersey Devils   
1     05/10/2024      New Jersey Devils          3       Buffalo Sabres   
2     08/10/2024         Seattle Kraken          2      St. Louis Blues   
3     08/10/2024       Florida Panthers          6        Boston Bruins   
4     09/10/2024       Utah Hockey Club          5   Chicago Blackhawks   
...          ...                    ...        ...                  ...   
1307  17/04/2025    Toronto Maple Leafs       None    Detroit Red Wings   
1308  17/04/2025        Ottawa Senators       None  Carolina Hurricanes   
1309  17/04/2025       New York Rangers       None  Tampa Bay Lightning   
1310  17/04/2025    Pittsburgh Penguins       None  Washington Capitals   
1311  17/04/2025  Columbus Blue Jackets       None   New 

In [15]:
# Mapping for team name normalization
team_name_mapping = {
    "Montréal Canadiens": "Montreal Canadiens",  # Normalize Montréal to Montreal
    # Add more mappings here if needed
}

# Normalize team names in Home Team and Away Team
df_game["Home Team"] = df_game["Home Team"].replace(team_name_mapping)
df_game["Away Team"] = df_game["Away Team"].replace(team_name_mapping)


In [16]:

# Rename 'Team' to 'Home Team' and merge to get Home Team Rank
df_game = df_game.merge(
    df_rank.rename(columns={"Team": "Home Team", "Rank": "Home Team Rank"}),
    on="Home Team",
    how="left"
)

# Rename 'Team' to 'Away Team' and merge to get Away Team Rank
df_game = df_game.merge(
    df_rank.rename(columns={"Team": "Away Team", "Rank": "Away Team Rank"}),
    on="Away Team",
    how="left"
)

# Final format with added ranking columns
final_columns = [
    "Date", "Home Team", "Home Score", "Away Team", "Away Score",
    "Home Team Rank", "Away Team Rank", "Home Win"
]
df_final = df_game[final_columns]

# Output the final DataFrame
print(df_final)

# Optionally save the result to a CSV
df_final.to_csv("merged_game_data.csv", index=False)


            Date              Home Team Home Score            Away Team  \
0     04/10/2024         Buffalo Sabres          1    New Jersey Devils   
1     05/10/2024      New Jersey Devils          3       Buffalo Sabres   
2     08/10/2024         Seattle Kraken          2      St. Louis Blues   
3     08/10/2024       Florida Panthers          6        Boston Bruins   
4     09/10/2024       Utah Hockey Club          5   Chicago Blackhawks   
...          ...                    ...        ...                  ...   
1307  17/04/2025    Toronto Maple Leafs       None    Detroit Red Wings   
1308  17/04/2025        Ottawa Senators       None  Carolina Hurricanes   
1309  17/04/2025       New York Rangers       None  Tampa Bay Lightning   
1310  17/04/2025    Pittsburgh Penguins       None  Washington Capitals   
1311  17/04/2025  Columbus Blue Jackets       None   New York Islanders   

     Away Score  Home Team Rank  Away Team Rank Home Win  
0             4              29         

In [18]:

# Filter out unplayed games
df_final = df_final.dropna(subset=["Home Score", "Away Score"])

# Convert scores and dates to appropriate data types
df_final["Date"] = pd.to_datetime(df_final["Date"], format="%d/%m/%Y")
df_final["Home Score"] = df_final["Home Score"].astype(int)
df_final["Away Score"] = df_final["Away Score"].astype(int)
df_final["Home Win"] = df_final["Home Win"].astype(bool)

# Sort by Date for chronological calculations
df_final = df_final.sort_values(by="Date").reset_index(drop=True)

# Helper function to calculate the last 10 games stats
def calculate_last_10_stats(df, team_column, win_column):
    stats = []
    for i, row in df.iterrows():
        team = row[team_column]
        past_games = df.iloc[:i]  # All games before the current one
        recent_games = past_games[(past_games["Home Team"] == team) | (past_games["Away Team"] == team)].tail(10)
        wins = (
            (recent_games["Home Team"] == team) & (recent_games["Home Win"])
        ).sum() + (
            (recent_games["Away Team"] == team) & (~recent_games["Home Win"])
        ).sum()
        stats.append(wins)
    return stats

# Add last 10 wins for Home and Away teams
df_final["Home Last 10 Wins"] = calculate_last_10_stats(df_final, "Home Team", "Home Win")
df_final["Away Last 10 Wins"] = calculate_last_10_stats(df_final, "Away Team", "Home Win")

# Add whether the team played the day before
def calculate_played_yesterday(df, team_column):
    played_yesterday = []
    for i, row in df.iterrows():
        team = row[team_column]
        current_date = row["Date"]
        past_games = df.iloc[:i]
        played = past_games[
            ((past_games["Home Team"] == team) | (past_games["Away Team"] == team))
            & (past_games["Date"] == current_date - pd.Timedelta(days=1))
        ]
        played_yesterday.append(len(played) > 0)
    return played_yesterday

df_final["Home Played Yesterday"] = calculate_played_yesterday(df_final, "Home Team")
df_final["Away Played Yesterday"] = calculate_played_yesterday(df_final, "Away Team")

# Add win rate for Home and Away teams
def calculate_win_rate(df, team_column, is_home_column):
    win_rate = []
    for i, row in df.iterrows():
        team = row[team_column]
        past_games = df.iloc[:i]
        if is_home_column:
            relevant_games = past_games[past_games["Home Team"] == team]
            wins = relevant_games["Home Win"].sum()
        else:
            relevant_games = past_games[past_games["Away Team"] == team]
            wins = (~relevant_games["Home Win"]).sum()
        win_rate.append(wins / len(relevant_games) if len(relevant_games) > 0 else 0)
    return win_rate

df_final["Home Win Rate"] = calculate_win_rate(df_final, "Home Team", is_home_column=True)
df_final["Away Win Rate"] = calculate_win_rate(df_final, "Away Team", is_home_column=False)

def calculate_overall_win_streak_excluding_current(df):
    streak = {}  # Dictionary to track current streak for each team
    overall_streak = []  # List to store the streak for each row

    for _, row in df.iterrows():
        # Get the teams
        home_team = row["Home Team"]
        away_team = row["Away Team"]

        # Append the current streaks (BEFORE updating for the current game)
        overall_streak.append((streak.get(home_team, 0), streak.get(away_team, 0)))

        # Initialize streak counters for both teams if not already done
        if home_team not in streak:
            streak[home_team] = 0
        if away_team not in streak:
            streak[away_team] = 0

        # Update streak for home team
        if row["Home Win"]:  # Home team won
            streak[home_team] += 1
        else:  # Home team lost
            streak[home_team] = 0

        # Update streak for away team
        if not row["Home Win"]:  # Away team won
            streak[away_team] += 1
        else:  # Away team lost
            streak[away_team] = 0

    # Return separate streak lists for home and away teams
    home_streaks, away_streaks = zip(*overall_streak)
    return list(home_streaks), list(away_streaks)


# Apply the modified function
home_streaks, away_streaks = calculate_overall_win_streak_excluding_current(df_final)

# Update the DataFrame with the adjusted streaks
df_final["Home Team Overall Win Streak Before Game"] = home_streaks
df_final["Away Team Overall Win Streak Before Game"] = away_streaks

# Function to calculate the average goal differential for the last 10 games
def calculate_avg_goal_differential(df, team_column, score_column, opponent_score_column):
    goal_differential = []
    window = 10  # Fixed window size to the last 10 games
    for i, row in df.iterrows():
        team = row[team_column]
        past_games = df.iloc[:i]  # All games before the current one
        relevant_games = past_games[
            (past_games["Home Team"] == team) | (past_games["Away Team"] == team)
        ].tail(window)

        # Calculate goal differential for relevant games
        gd = 0
        for _, game in relevant_games.iterrows():
            if game["Home Team"] == team:
                gd += game[score_column] - game[opponent_score_column]
            else:
                gd += game[opponent_score_column] - game[score_column]

        # Calculate the average
        avg_gd = gd / len(relevant_games) if len(relevant_games) > 0 else 0
        goal_differential.append(avg_gd)
    return goal_differential

# Function to calculate the average opponent rank for the last 10 games
def calculate_avg_opponent_rank(df, team_column, opponent_rank_column, window=10):
    avg_opponent_rank = []  # List to store the average rank for each game

    for i, row in df.iterrows():
        # Use the get_relevant_games function to retrieve the relevant games for the current game
        relevant_games = get_relevant_games(df, i, team_column, window)

        # Calculate the total rank of opponents
        total_rank = 0
        for _, game in relevant_games.iterrows():
            if game["Home Team"] == row[team_column]:  # If the team is the Home Team
                total_rank += game["Away Team Rank"]
            else:  # If the team is the Away Team
                total_rank += game["Home Team Rank"]

        # Calculate the average opponent rank
        avg_rank = total_rank / len(relevant_games) if len(relevant_games) > 0 else np.nan
        avg_opponent_rank.append(avg_rank)

    return avg_opponent_rank

def get_relevant_games(df, index, team_column, window):
    """
    Retrieve the last `window` games for a team prior to the current game.

    Parameters:
        df (pd.DataFrame): DataFrame containing the games.
        index (int): Index of the current game.
        team_column (str): Column name indicating the team.
        window (int): Number of past games to consider.

    Returns:
        pd.DataFrame: DataFrame of relevant games.
    """
    team = df.iloc[index][team_column]
    past_games = df.iloc[:index]  # All games before the current game
    relevant_games = past_games[
        (past_games["Home Team"] == team) | (past_games["Away Team"] == team)
    ].tail(window)
    return relevant_games

# Re-run the calculate_avg_opponent_rank function
df_final["Home Opponent Strength"] = calculate_avg_opponent_rank(
    df_final, "Home Team", "Away Team Rank", window=10
)
df_final["Away Opponent Strength"] = calculate_avg_opponent_rank(
    df_final, "Away Team", "Home Team Rank", window=10
)

# Debug: Print the dataset to verify results
print(df_final[["Date", "Home Team", "Away Team", "Home Opponent Strength", "Away Opponent Strength"]].tail(10))

# Final DataFrame with all calculated columns
print(df_final)

# Save to CSV
df_final.to_csv("enhanced_game_data.csv", index=False)


          Date             Home Team            Away Team  \
680 2025-01-12   Pittsburgh Penguins  Tampa Bay Lightning   
681 2025-01-12       Ottawa Senators         Dallas Stars   
682 2025-01-12        Calgary Flames    Los Angeles Kings   
683 2025-01-12       San Jose Sharks       Minnesota Wild   
684 2025-01-12  Vegas Golden Knights     New York Rangers   
685 2025-01-12   Carolina Hurricanes        Anaheim Ducks   
686 2025-01-13  Vegas Golden Knights       Minnesota Wild   
687 2025-01-14   Philadelphia Flyers     Florida Panthers   
688 2025-01-14    Chicago Blackhawks       Calgary Flames   
689 2025-01-14       Edmonton Oilers    Los Angeles Kings   

     Home Opponent Strength  Away Opponent Strength  
680                    16.3                    16.9  
681                    13.8                    22.5  
682                    20.7                    14.9  
683                    11.7                    15.4  
684                    24.4                    11.4  
685 

In [19]:
# Function to calculate rest days since the last game (checking both Home and Away appearances)
def calculate_rest_days_any_game(df, team_column):
    last_played = {}  # Dictionary to track the last game date for each team
    rest_days = []

    for _, row in df.iterrows():
        team = row[team_column]
        game_date = row["Date"]

        # Determine the rest days by looking at both Home and Away appearances
        if team in last_played:
            rest_days.append((game_date - last_played[team]).days)
        else:
            rest_days.append(np.nan)  # If the team hasn't played yet

        # Update the last played date for the team (check both Home Team and Away Team)
        last_played[team] = game_date

        # Update last played for both home and away games
        for opponent in ["Home Team", "Away Team"]:
            if row[opponent] == team:
                last_played[team] = game_date

    return rest_days

# Ensure the Date column is in datetime format
df_final["Date"] = pd.to_datetime(df_final["Date"])

# Calculate rest days for both Home and Away teams by considering all games (home or away)
df_final["Home Rest Days"] = calculate_rest_days_any_game(df_final, "Home Team")
df_final["Away Rest Days"] = calculate_rest_days_any_game(df_final, "Away Team")


In [20]:
# Display the last 10 observations
print(df_final.tail(10))


          Date             Home Team  Home Score            Away Team  \
680 2025-01-12   Pittsburgh Penguins           2  Tampa Bay Lightning   
681 2025-01-12       Ottawa Senators           3         Dallas Stars   
682 2025-01-12        Calgary Flames           2    Los Angeles Kings   
683 2025-01-12       San Jose Sharks           1       Minnesota Wild   
684 2025-01-12  Vegas Golden Knights           1     New York Rangers   
685 2025-01-12   Carolina Hurricanes           2        Anaheim Ducks   
686 2025-01-13  Vegas Golden Knights           4       Minnesota Wild   
687 2025-01-14   Philadelphia Flyers           4     Florida Panthers   
688 2025-01-14    Chicago Blackhawks           2       Calgary Flames   
689 2025-01-14       Edmonton Oilers           1    Los Angeles Kings   

     Away Score  Home Team Rank  Away Team Rank  Home Win  Home Last 10 Wins  \
680           5              19              14     False                  3   
681           2              18     

In [21]:

results = df_final
results.head()

,Date,Home Team,Home Score,Away Team,Away Score,Home Team Rank,Away Team Rank,Home Win,Home Last 10 Wins,Away Last 10 Wins,Home Played Yesterday,Away Played Yesterday,Home Win Rate,Away Win Rate,Home Team Overall Win Streak Before Game,Away Team Overall Win Streak Before Game,Home Opponent Strength,Away Opponent Strength,Home Rest Days,Away Rest Days
0,2024-10-04,Buffalo Sabres,1,New Jersey Devils,4,29,7,False,0,0,False,False,0.0,0.0,0,0,NaN,NaN,NaN,NaN
1,2024-10-05,New Jersey Devils,3,Buffalo Sabres,1,7,29,True,1,0,True,True,0.0,0.0,1,0,29.0,7.0,NaN,NaN
2,2024-10-08,Seattle Kraken,2,St. Louis Blues,3,28,20,False,0,0,False,False,0.0,0.0,0,0,NaN,NaN,NaN,NaN
3,2024-10-08,Florida Panthers,6,Boston Bruins,4,11,16,True,0,0,False,False,0.0,0.0,0,0,NaN,NaN,NaN,NaN
4,2024-10-09,Utah Hockey Club,5,Chicago Blackhawks,2,24,32,True,0,0,False,False,0.0,0.0,0,0,NaN,NaN,NaN,NaN


In [22]:

scorer = make_scorer(f1_score, pos_label=None, average='weighted')

# Define y_true based on the Home Win column
y_true = df_final["Home Win"].astype(int)  # Convert True/False to 1/0

# Create a dummy prediction where all predictions are 1 (Home Win = True)
y_pred = [1] * len(y_true)

# Calculate the weighted F1 score
f1 = f1_score(y_true, y_pred, pos_label=None, average="weighted")

# Print the F1 score
print(f"F1 Score: {f1:.4f}")

F1 Score: 0.3794


In [23]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Load your dataset (assuming it's stored in `df_final`)
# Ensure the target variable is properly defined
target = "Home Win"  # Define the target column
features = [
    "Home Team Rank",
    "Away Team Rank",
    "Home Last 10 Wins",
    "Away Last 10 Wins",
    "Home Played Yesterday",
    "Away Played Yesterday",
    "Home Win Rate",
    "Away Win Rate",
    "Home Team Overall Win Streak Before Game",
    "Away Team Overall Win Streak Before Game",
    "Home Opponent Strength","Away Opponent Strength"
    
]  # Select only numerical and categorical features for the model

# Prepare the dataset
X = df_final[features]
y = df_final[target].astype(int)  # Convert True/False to 1/0

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average="weighted")
print(f"F1 Score of the model: {f1:.4f}")

# Get feature importances from the Random Forest
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({"Feature": features, "Importance": importances})
feature_importance_df = feature_importance_df.sort_values(by="Importance", ascending=False)
print("Feature Importance from Random Forest:")
print(feature_importance_df)

# Calculate permutation importance
perm_importance = permutation_importance(model, X_test, y_test, scoring="f1_weighted", random_state=42)
perm_importance_df = pd.DataFrame({
    "Feature": features,
    "Importance": perm_importance.importances_mean
}).sort_values(by="Importance", ascending=False)
print("\nPermutation Importance:")
print(perm_importance_df)
#"Away Opponent Strength"
# Save feature importances to CSV files
feature_importance_df.to_csv("feature_importance_random_forest.csv", index=False)
perm_importance_df.to_csv("feature_importance_permutation.csv", index=False)

F1 Score of the model: 0.6217
Feature Importance from Random Forest:
                                     Feature  Importance
1                             Away Team Rank    0.139835
0                             Home Team Rank    0.138020
10                    Home Opponent Strength    0.130379
11                    Away Opponent Strength    0.124784
6                              Home Win Rate    0.115270
7                              Away Win Rate    0.102848
2                          Home Last 10 Wins    0.059219
3                          Away Last 10 Wins    0.058856
8   Home Team Overall Win Streak Before Game    0.046197
9   Away Team Overall Win Streak Before Game    0.045057
5                      Away Played Yesterday    0.021285
4                      Home Played Yesterday    0.018250

Permutation Importance:
                                     Feature  Importance
0                             Home Team Rank    0.077644
1                             Away Team Rank    0.0

In [24]:
# Retain only the important features with positive permutation importance
selected_features = [
    "Home Team Rank",               # High importance in both analyses
    "Away Team Rank",               # High importance in both analyses
    "Home Opponent Strength",       # High in Random Forest
    "Away Opponent Strength",       # High in Random Forest
    "Home Win Rate",                # High in both analyses
    "Away Win Rate",                # High in both analyses
    "Home Played Yesterday",        # Appears in both
    "Away Played Yesterday",        # Appears in both
    "Home Last 10 Wins",            # Included for diversity despite low score
    "Away Last 10 Wins",            # Included for diversity despite low score
    "Home Team Overall Win Streak Before Game",  # Decent in permutation importance
    "Away Team Overall Win Streak Before Game"   # Decent in permutation importance
]

X_selected = df_final[selected_features]
y = df_final["Home Win"].astype(int)

# Train-test split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Train the Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average="weighted")
print(f"F1 Score with selected features: {f1:.4f}")


F1 Score with selected features: 0.5870


In [25]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoding = LabelEncoder() # encoding object has LabelEncoder ready to use
encoding.fit(results["Home Team"].values) # give each team a number to identify them by

LabelEncoder()

In [26]:
home_teams = encoding.transform(results["Home Team"].values)
visitor_teams = encoding.transform(results["Away Team"].values)

# vstack allows you to stack a sequence of arrays vertically - in this case we do it to make easy comparisons
X_teams = np.vstack([home_teams, visitor_teams]).T

X_teams[:5], X_teams.shape

(array([[ 2, 16],
        [16,  2],
        [23, 24],
        [11,  1],
        [27,  5]]),
 (690, 2))

In [27]:
# Prepare the One Hot Encoder
onehot = OneHotEncoder()
X_teams = onehot.fit_transform(X_teams).toarray()


In [28]:

X_teams.shape # see matrix dimensions

(690, 64)

In [29]:
# Lets build our model using the Decision Tree Classifier
tree_clf = DecisionTreeClassifier(random_state=10)
scores = cross_val_score(tree_clf, X_teams, y_true, scoring=scorer)

# Note: The only feature is the encoder we just made
print("The F1 score using encoded teams is the following: {0:.4f}".format(np.mean(scores)))

The F1 score using encoded teams is the following: 0.5353


In [30]:

forest_clf = RandomForestClassifier(random_state=10)
scores = cross_val_score(forest_clf, X_teams, y_true, scoring=scorer)
print("The F1 score using Random Forest Classifier is the following: {0:.4f}".format(np.mean(scores)))

The F1 score using Random Forest Classifier is the following: 0.5691


In [31]:
# Define the parameter grid with valid values for max_features
parameter_space = {
    "max_features": [2, 10, 50, 'sqrt', 'log2', None],  # Replace 'auto' with valid options
    "n_estimators": [50, 100, 200],
    "criterion": ["gini", "entropy"],
    "min_samples_leaf": [1, 2, 4, 6],
}

# Initialize the Random Forest model and the scoring function
forest_clf = RandomForestClassifier(random_state=10)
scorer = make_scorer(f1_score, pos_label=None, average='weighted')

# Set up GridSearchCV
grid = GridSearchCV(forest_clf, parameter_space, scoring=scorer, error_score='raise')

# Fit the model
try:
    grid.fit(X_teams, y_true)
except Exception as e:
    print(f"An error occurred during grid search: {e}")

# View results if the fit is successful
print("F1: {0:.4f}".format(grid.best_score_))
print("Best Parameters:", grid.best_params_)
print("Best Estimator:", grid.best_estimator_)

F1: 0.6027
Best Parameters: {'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 1, 'n_estimators': 100}
Best Estimator: RandomForestClassifier(max_features='log2', random_state=10)


In [33]:
# Extract the features
X_homehigher = results[selected_features].values

# Combine with additional features (e.g., one-hot encoded team features)
X_all = np.hstack([X_homehigher, X_teams])
print("Feature matrix shape:", X_all.shape)

# Define the scorer (e.g., F1 score)
scorer = make_scorer(f1_score, average="weighted")

# Decision Tree Classifier
tree_clf = DecisionTreeClassifier(random_state=10)
tree_scores = cross_val_score(tree_clf, X_all, y_true, scoring=scorer, cv=5)
print("F1 score for Decision Tree: {0:.4f}".format(np.mean(tree_scores)))

# Random Forest Classifier
rf_clf = RandomForestClassifier(random_state=10, n_estimators=100, max_depth=None)
rf_scores = cross_val_score(rf_clf, X_all, y_true, scoring=scorer, cv=5)
print("F1 score for Random Forest: {0:.4f}".format(np.mean(rf_scores)))

Feature matrix shape: (690, 76)
F1 score for Decision Tree: 0.5253
F1 score for Random Forest: 0.5758


In [34]:
# Define the parameter grid with valid values for max_features
parameter_space = {
    "max_features": [2, 10, 50, 'sqrt', 'log2', None],  # Replaced 'auto' with valid options
    "n_estimators": [50, 100, 200],
    "criterion": ["gini", "entropy"],
    "min_samples_leaf": [1, 2, 4, 6],
}

# Initialize the Random Forest model and scoring function
forest_clf = RandomForestClassifier(random_state=10)
scorer = make_scorer(f1_score, pos_label=None, average='weighted')

# Set up GridSearchCV
grid = GridSearchCV(forest_clf, parameter_space, scoring=scorer, error_score='raise')  # Added error_score='raise'

# Fit the model
try:
    grid.fit(X_all, y_true)
except Exception as e:
    print(f"An error occurred during grid search: {e}")

# Display results if the fit is successful
print("F1 score: {0:.4f}".format(grid.best_score_))
print("Best Parameters:", grid.best_params_)
print("Best Estimator:", grid.best_estimator_)

F1 score: 0.6222
Best Parameters: {'criterion': 'entropy', 'max_features': 50, 'min_samples_leaf': 2, 'n_estimators': 100}
Best Estimator: RandomForestClassifier(criterion='entropy', max_features=50, min_samples_leaf=2,
                       random_state=10)


In [40]:

selected_features = [
    "Home Team Rank",               # High importance in both analyses
    "Away Team Rank",               # High importance in both analyses
    "Home Opponent Strength",       # High in Random Forest
    "Away Opponent Strength",       # High in Random Forest
    "Home Win Rate",                # High in both analyses
    "Away Win Rate",                # High in both analyses
    "Home Played Yesterday",        # Appears in both
    "Away Played Yesterday",        # Appears in both
    "Home Last 10 Wins",            # Included for diversity despite low score
    "Away Last 10 Wins",            # Included for diversity despite low score
    "Home Team Overall Win Streak Before Game",  # Decent in permutation importance
    "Away Team Overall Win Streak Before Game"   # Decent in permutation importance
]

# List of all teams (from encoding.classes_)
teams = list(encoding.classes_)

# Display the list of teams
print("Available Teams:")
for i, team in enumerate(teams):
    print(f"{i + 1}: {team}")

# Let the user select home and away teams by number
home_team_index = int(input("Select the home team by number: ")) - 1
away_team_index = int(input("Select the away team by number: ")) - 1

# Get the team names based on user selection
home_team = teams[home_team_index]
away_team = teams[away_team_index]

print(f"Home Team: {home_team}")
print(f"Away Team: {away_team}")

# Retrieve the selected features for the teams
input_data = results[
    (results["Home Team"] == home_team) & (results["Away Team"] == away_team)
][selected_features]

if input_data.empty:
    print("No matching game found for the selected teams in the dataset.")
else:
    # Create one-hot encoding for the teams
    one_hot_home = [1 if team == home_team else 0 for team in teams]
    one_hot_away = [1 if team == away_team else 0 for team in teams]

    # Combine features into a single input vector
    combined_features = np.hstack([
        input_data.iloc[0].values,  # Selected features
        one_hot_home,  # One-hot encoded home team
        one_hot_away   # One-hot encoded away team
    ])

    # Ensure the input vector matches the model's expected format
    input_vector = combined_features.reshape(1, -1)  # Reshape for prediction

    # Make the prediction using the best model
    predicted_prob = grid.best_estimator_.predict_proba(input_vector)

    # Extract probabilities for home and away teams
    home_team_prob = predicted_prob[0][1]  # Probability of home team winning
    away_team_prob = predicted_prob[0][0]  # Probability of away team winning

    # Input: Gambling site odds
    home_team_odds = float(input(f"Enter the odds for {home_team}: "))
    away_team_odds = float(input(f"Enter the odds for {away_team}: "))

    # Kelly Criterion Formula
    def kelly_criterion(prob, odds):
        b = odds - 1
        q = 1 - prob
        return max(0, (b * prob - q) / b)  # Return 0 if Kelly fraction is negative

    # Calculate Kelly Criterion for both teams
    home_kelly = kelly_criterion(home_team_prob, home_team_odds)
    away_kelly = kelly_criterion(away_team_prob, away_team_odds)

    # Determine the best bet
    if home_kelly > away_kelly:
        print(f"Bet on {home_team}. Kelly Criterion suggests betting {home_kelly:.2%} of your bankroll.")
    elif away_kelly > home_kelly:
        print(f"Bet on {away_team}. Kelly Criterion suggests betting {away_kelly:.2%} of your bankroll.")
    else:
        print("Neither bet has a positive expected value according to the Kelly Criterion.")

    # Output additional details
    print(f"Home Team: {home_team}, Probability: {home_team_prob:.2%}, Odds: {home_team_odds}")
    print(f"Away Team: {away_team}, Probability: {away_team_prob:.2%}, Odds: {away_team_odds}")
    print(f"Kelly Criterion (Home): {home_kelly:.2%}")
    print(f"Kelly Criterion (Away): {away_kelly:.2%}")
    

Available Teams:
1: Anaheim Ducks
2: Boston Bruins
3: Buffalo Sabres
4: Calgary Flames
5: Carolina Hurricanes
6: Chicago Blackhawks
7: Colorado Avalanche
8: Columbus Blue Jackets
9: Dallas Stars
10: Detroit Red Wings
11: Edmonton Oilers
12: Florida Panthers
13: Los Angeles Kings
14: Minnesota Wild
15: Montreal Canadiens
16: Nashville Predators
17: New Jersey Devils
18: New York Islanders
19: New York Rangers
20: Ottawa Senators
21: Philadelphia Flyers
22: Pittsburgh Penguins
23: San Jose Sharks
24: Seattle Kraken
25: St. Louis Blues
26: Tampa Bay Lightning
27: Toronto Maple Leafs
28: Utah Hockey Club
29: Vancouver Canucks
30: Vegas Golden Knights
31: Washington Capitals
32: Winnipeg Jets


Select the home team by number:  28
Select the away team by number:  15


Home Team: Utah Hockey Club
Away Team: Montreal Canadiens
No matching game found for the selected teams in the dataset.
